In [1]:
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata

file = r"C:\Users\Daniel\Desktop\Bilder Papa\VID_20220929_144049.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20210902_183602.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20210917_115704.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20220929_090345.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20220929_090439.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20220929_090755.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20220929_144049.mp4"
file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20221002_102055.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20221103_060107.mp4"
# file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20221105_124004.mp4"
file = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\VID_20221105_124104.mp4"
# file = r
# file = r

with createParser(file) as parser:
    metadata = extractMetadata(parser)
    for line in metadata.exportPlaintext():
        print(line)
        # if "- Creation date: " in line:
        #     print(line.split("- Creation date: ")[1])
        #     print(line)

Metadata:
- Duration: 18 sec 687 ms
- Image width: 1920 pixels
- Image height: 1080 pixels
- Creation date: 2022-11-05 12:41:04
- Last modification: 2022-11-05 12:41:04
- Comment: Play speed: 100.0%
- Comment: User volume: 100.0%
- MIME type: video/mp4
- Endianness: Big endian


In [ ]:
import time
import os.path

time.ctime(os.path.getctime(file))

In [ ]:
import os

# Replace 'path/to/file.mp4' with the actual path to your video file
file_path = file

# Get the file's metadata using the os.stat function
metadata = os.stat(file_path)

# The creation timestamp is stored in the st_ctime attribute of the metadata
creation_timestamp = metadata.st_ctime

# Print the creation timestamp
print(creation_timestamp)

import datetime

# Convert the Unix timestamp to a datetime object
dt = datetime.datetime.fromtimestamp(creation_timestamp)

# Format the datetime object as a string in the desired format
formatted_timestamp = dt.strftime('%Y-%m-%d %H:%M:%S')

# Print the formatted timestamp
print(formatted_timestamp)

In [ ]:
import pymediainfo

# Replace 'path/to/file.mp4' with the actual path to your video file
file_path = file

# Extract the metadata from the video file using pymediainfo
metadata = pymediainfo.MediaInfo.parse(file_path).to_data()

pass

# The capture timestamp is stored in the 'record_date' field of the metadata
capture_timestamp = metadata['tracks'][0]['record_date']

# Print the capture timestamp
print(capture_timestamp)

In [ ]:
import ffmpeg

# Replace 'path/to/file.mp4' with the actual path to your video file
file_path = file

# Use the ffmpeg.probe function to get the metadata for the video file
metadata = ffmpeg.probe(file_path)

# The timestamp of the capture is stored in the 'creation_time' field of the metadata
creation_time = metadata['format']['tags']['creation_time']

# Print the creation time
print(creation_time)


In [ ]:
import pymediainfo
import sys

media_info = pymediainfo.MediaInfo.parse(file)
print(media_info)
#For the first track - otherwise iterate over each track
print(media_info.general_tracks[0].file_creation_date)

In [ ]:
import pymediainfo

# Replace 'path/to/file.mp4' with the actual path to your video file
file_path = file

# Extract the metadata from the video file using pymediainfo
metadata = pymediainfo.MediaInfo.parse(file_path).to_data()

pass

# The capture timestamp is stored in the 'record_date' field of the metadata
capture_timestamp = metadata['tracks'][0]['record_date']

# Print the capture timestamp
print(capture_timestamp)

In [ ]:
def get_mov_timestamps(filename):
    ''' Get the creation and modification date-time from .mov metadata.

        Returns None if a value is not available.
    '''
    from datetime import datetime as DateTime
    import struct

    ATOM_HEADER_SIZE = 8
    # difference between Unix epoch and QuickTime epoch, in seconds
    EPOCH_ADJUSTER = 2082844800

    creation_time = modification_time = None

    # search for moov item
    with open(filename, "rb") as f:
        while True:
            atom_header = f.read(ATOM_HEADER_SIZE)
            #~ print('atom header:', atom_header)  # debug purposes
            if atom_header[4:8] == b'moov':
                break  # found
            else:
                atom_size = struct.unpack('>I', atom_header[0:4])[0]
                f.seek(atom_size - 8, 1)

        # found 'moov', look for 'mvhd' and timestamps
        atom_header = f.read(ATOM_HEADER_SIZE)
        if atom_header[4:8] == b'cmov':
            raise RuntimeError('moov atom is compressed')
        elif atom_header[4:8] != b'mvhd':
            raise RuntimeError('expected to find "mvhd" header.')
        else:
            f.seek(4, 1)
            creation_time = struct.unpack('>I', f.read(4))[0] - EPOCH_ADJUSTER
            creation_time = DateTime.fromtimestamp(creation_time)
            if creation_time.year < 1990:  # invalid or censored data
                creation_time = None

            modification_time = struct.unpack('>I', f.read(4))[0] - EPOCH_ADJUSTER
            modification_time = DateTime.fromtimestamp(modification_time)
            if modification_time.year < 1990:  # invalid or censored data
                modification_time = None

    return creation_time, modification_time

creation_time, modification_time = get_mov_timestamps(file)
print(creation_time)
print(modification_time)


In [3]:
import quicktimeparser.parse as qtparse

qt = qtparse.Mov(file)
qt.parse()

#retrieve the creation-time as a string (the actual creation time, not the file-system creation time)
date = qt.metadata["creation time"]

#traverse all key-value pairs of the metadata
for key in qt.metadata.keys():
  print(key+": "+str(qt.metadata[key]))


KeyError: 'creation time'

In [10]:
import subprocess

def get_exif_creation_dates_video(path):
    EXIFTOOL_DATE_TAG_VIDEOS = "Create Date"
    EXIF_DATE_FORMAT = "%Y-%m-%d %H:%M:%S"
    absolute_path = os.path.join( os.getcwd(), path )

    process = subprocess.Popen(["exiftool", absolute_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    lines = out.decode("utf-8").split("\n")
    for l in lines:
        if EXIFTOOL_DATE_TAG_VIDEOS in str(l):
                datetime_str = str(l.split(" : ")[1].strip())
                dt = datetime.strptime(datetime_str, EXIF_DATE_FORMAT)
                print(dt) #you will get 3 dates: Create Date, Track Create Date and Media Create Date 

file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"
get_exif_creation_dates_video(file)
# print(file)

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden

In [14]:
import exifread

file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"
f = open(file, "rb")
tags = exifread.process_file(f)
print(tags)

File format not recognized.


{}


In [16]:
EXIFTOOL_DATE_TAG_VIDEOS = "Create Date"
EXIF_DATE_FORMAT = "%Y:%m:%d %H:%M:%S"
file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"

cmd = "'C:\Users\Daniel\Desktop\Bilder Papa - Kopie\exiftool-12.52\exiftool(-k).exe' '%s'" % file
output = subprocess.check_output(cmd, shell=True)
lines = output.decode("ascii").split("\n")
for l in lines:
    if EXIFTOOL_DATE_TAG_VIDEOS in l:
            datetime_str = l.split(" : ")[1]
            print(datetime.datetime.strptime(datetime_str,
                                             EXIF_DATE_FORMAT))

# exiftool(-k).exe



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (1971335789.py, line 5)

In [20]:
def get_exif_creation_dates_video(path):
    EXIFTOOL_DATE_TAG_VIDEOS = "Create Date"
    EXIF_DATE_FORMAT = "%Y-%m-%d %H:%M:%S"
    absolute_path = os.path.join( os.getcwd(), path )

    process = subprocess.Popen([r'C:\Users\Daniel\Desktop\Bilder Papa - Kopie\exiftool-12.52\exiftool(-k).exe', absolute_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    lines = out.decode("utf-8").split("\n")
    for l in lines:
        if EXIFTOOL_DATE_TAG_VIDEOS in str(l):
                datetime_str = str(l.split(" : ")[1].strip())
                dt = datetime.strptime(datetime_str, EXIF_DATE_FORMAT)
                print(dt) #you will get 3 dates: Create Date, Track Create Date and Media Create Date 

file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"
get_exif_creation_dates_video(file)

In [6]:
import ffmpeg
import sys
from pprint import pprint # for printing Python dictionaries in a human-readable way

file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"

# read the audio/video file from the command line arguments
media_file = file
# uses ffprobe command to extract all possible metadata from the media file
ffmpeg.probe(media_file)["streams"]


FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden

In [8]:
import ffmpeg
file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"
vid = ffmpeg.probe(file)
print(vid['streams'])

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden

In [15]:

import subprocess
import datetime

def get_media_properties(filename):
    
    result = subprocess.Popen(['hachoir-metadata', filename, '--raw', '--level=3'],
        stdout = subprocess.PIPE, stderr = subprocess.STDOUT)

    results = result.stdout.read().decode('utf-8').split('\r\n')
    print(results)
    print(result)

    properties = {}

    # for item in results:

    #     if item.startswith('- duration: '):
    #         duration = item.lstrip('- duration: ')
    #         if '.' in duration:
    #             t = datetime.datetime.strptime(item.lstrip('- duration: '), '%H:%M:%S.%f')
    #         else:
    #             t = datetime.datetime.strptime(item.lstrip('- duration: '), '%H:%M:%S')
    #         seconds = (t.microsecond / 1e6) + t.second + (t.minute * 60) + (t.hour * 3600)
    #         properties['duration'] = round(seconds)

    #     if item.startswith('- width: '):
    #         properties['width'] = int(item.lstrip('- width: '))

    #     if item.startswith('- height: '):
    #         properties['height'] = int(item.lstrip('- height: '))

    # return properties

get_media_properties(file)

['Metadata:', '- duration: 0:00:18.687600', '- width: 1920', '- height: 1080', '']
<Popen: returncode: None args: ['hachoir-metadata', 'C:/Users/Daniel/Desktop...>


In [19]:
# Python3 program to illustrate
# accessing of audio metadata
# using tinytag library
  
# Import Tinytag method from
# tinytag library
from tinytag import TinyTag
  
# Pass the filename into the
# Tinytag.get() method and store
# the result in audio variable
file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"
video = TinyTag.get(file)
  
# Use the attributes
# and Display
# Use the attributes
# and display
print("Title:" + video.title)
print("Artist: " + video.artist)
print("Genre:" + video.genre)
print("Year Released: " + video.year)
print("Bitrate:" + str(video.bitrate) + " kBits/s")
print("Composer: " + video.composer)
print("Filesize: " + str(video.filesize) + " bytes")
print("AlbumArtist: " + str(video.albumartist))
print("Duration: " + str(video.duration) + " seconds")
print("TrackTotal: " + str(video.track_total))

TypeError: can only concatenate str (not "NoneType") to str

In [1]:
# import the preinstalled module
import subprocess
# write the complete path of your video file along with its type for example mp4 
input_file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"
# write the complete path of the ExifTool in your device along with .exe at last
exe=r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\exiftool-12.52\exiftool(-k).exe"
# process is a variable
process=subprocess.Popen([exe, input_file],stdout=subprocess.PIPE,stderr=subprocess.STDOUT,universal_newlines=True)
# For the output
for output in process.stdout:
    print(output.strip()) # strip is used to remove unwanted spaces

-- press ENTER --


In [5]:
import exiftool

# # files = ["a.jpg", "b.png", "c.tif"]
# input_file = "C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4"

# with exiftool.ExifTool() as et:
#     metadata = et.get_metadata_batch(input_file)
# # for d in metadata:


import exiftool
f = open("C:/Users/Daniel/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4")
exiftool.get_metadata_batch(f)

AttributeError: module 'exiftool' has no attribute 'get_metadata_batch'

In [70]:
import subprocess
import os
import json
from datetime import datetime
from dateutil.parser import parse

class ExifTool(object):

    sentinel = "{ready}\r\n"

    def __init__(self, executable=r"exiftool.exe"):
        self.executable = executable

    def __enter__(self):
        self.process = subprocess.Popen(
            [self.executable, "-stay_open", "True",  "-@", "-"],
            stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        return self

    def  __exit__(self, exc_type, exc_value, traceback):
        self.process.stdin.write("-stay_open\nFalse\n")
        self.process.stdin.flush()

    def execute(self, *args):
        args = args + ("-execute\n",)

        self.process = subprocess.Popen(
                [self.executable, "-stay_open", "True",  "-@", "-"],
                universal_newlines=True,
                stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        self.process.stdin.write(str.join("\n", args))
        self.process.stdin.flush()
        output = ""
        fd = self.process.stdout.fileno()
        while not output.endswith(self.sentinel):
            output += os.read(fd, 4096).decode('utf-8')
        return output[:-len(self.sentinel)]

    def get_metadata(self, *filenames):
        return json.loads(self.execute("-G", "-j", "-n", *filenames))

with ExifTool() as e:
    # metadata = e.get_metadata("C:/Users/DanieCl/Desktop/Bilder Papa - Kopie/_/VID_20221105_124104.mp4")
    metadata = e.get_metadata(r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\renamed\renamed\VID_20221222_060107.mp4")

# print(metadata[0]["File:FileAccessDate"])
# print(metadata[0]["File:FileCreateDate"])
# print(metadata[0]["File:FileModifyDate"])
# print(parse(metadata[0]["File:FileModifyDate"]).replace(tzinfo=None))
# print(type(parse(metadata[0]["File:FileModifyDate"])))
print(parse(metadata[0]["File:FileModifyDate"]))
print(parse(metadata[0]["File:FileAccessDate"]))
print(parse(metadata[0]["File:FileCreateDate"]))
print(parse(metadata[0]["QuickTime:CreateDate"]))
print(parse(metadata[0]["QuickTime:ModifyDate"]))
print(parse(metadata[0]["QuickTime:TrackCreateDate"]))
print(parse(metadata[0]["QuickTime:TrackModifyDate"]))
print(parse(metadata[0]["QuickTime:MediaCreateDate"]))
print(parse(metadata[0]["QuickTime:MediaModifyDate"]))
# File:FileModifyDate
# File:FileAccessDate
# File:FileCreateDate
# QuickTime:CreateDate
# QuickTime:ModifyDate
# QuickTime:TrackCreateDate
# QuickTime:TrackModifyDate
# QuickTime:MediaCreateDate
# QuickTime:MediaModifyDate


# print(datetime(2022, 11, 22, 13, 41, 4, tzinfo="Europe/Berlin") == parse(metadata[0]["File:FileModifyDate"]))

2022-12-22 07:01:08+01:00
2022-12-22 23:39:02+01:00
2022-12-22 19:54:07+01:00
2022-12-22 06:01:07
2022-12-22 06:01:07
2022-12-22 06:01:07
2022-12-22 06:01:07
2022-12-22 06:01:07
2022-12-22 06:01:07


In [64]:
parse(metadata[0]["File:FileModifyDate"])

datetime.datetime(2022, 12, 22, 13, 41, 4, tzinfo=tzoffset(None, 3600))

In [58]:
import pytz
datetime(2022, 12, 22, 13, 41, 4).astimezone(pytz.timezone("Europe/Berlin")) == parse(metadata[0]["File:FileModifyDate"])
datetime(2022, 12, 22, 13, 41, 4).astimezone(pytz.timezone("Europe/Berlin")) == parse(metadata[0]["File:FileModifyDate"])

True

# -----------------

In [71]:
path = r"C:\Users\Daniel\Desktop\Bilder Papa - Kopie\_\renamed\renamed\renamed\IMG_20220615_120350.jpg"

In [73]:
from exif import Image
with open(path, "rb") as src:
    img = Image(src)        
    print(datetime.strptime(img.datetime, '%Y:%m:%d %H:%M:%S'))

2022-06-15 12:03:50
